In [1]:
suppressMessages(library("rwwa"))

In [2]:
overlay_dnorm <- function(ts, x = seq(50,100,0.1), ...) lines(x, dnorm(x, mean(ts), sd(ts)), ...)

# SSTs

## Choose distribution

In [48]:
nc = 3; dpi = 360
for (fnm in list.files("ts", pattern = "sst", full.names = T)) {
    
    mdl = strsplit(gsub(".csv","",fnm),"_")[[1]][3]
    df = read.csv(fnm)
    
    gmst = load_ts(paste0("ts/SGSAT2023.",mdl,"_r1i1p1f1.hssp585.dat"), col.names = c("year", "gmst"))
    df <- merge(gmst, df)
    
    png(paste0("fig/cmodels/dist_",substr(fnm,4,nchar(fnm)-4),".png"), width = dpi*nc, height = dpi); {
        prep_window(c(1,nc), oma = c(0,0,2,0))

        plot(df$year, df$jjas_1d, type = "l", ylab = "Mean SST (degC)", xlab = "", main = "Annual time series")
        lines(df$year, df$jjas_7d, col = "blue2")
        lines(df$year, df$jjas_m, col = "red3")
        legend("topleft", legend = c("1day", "7day", "Monthly"), lty = 1, col = c("black", "blue2", "red3"))

        plot(density(df$jjas_m), col = "red3", xlab = "Mean SST (degC)", main = "Distribution of temps")
        lines(density(df$jjas_7d), col = "blue2")
        lines(density(df$jjas_1d))
        legend("topright", legend = c("1day", "7day", "Monthly"), lty = 1, col = c("black", "blue2", "red3"))
    
        plot(density(resid(lm(jjas_m ~ gmst, df))), col = "red3", xlab = "Residual", main = "Residuals after GMST accounted for", ylim = c(0,1.5))
        overlay_dnorm(resid(lm(jjas_m ~ gmst, df)), col = "red3", x = seq(-1,1,0.01), lty = 2)
        lines(density(resid(lm(jjas_7d ~ gmst, df))), col = "blue2")
        overlay_dnorm(resid(lm(jjas_7d ~ gmst, df)), col = "blue2", x = seq(-1,1,0.01), lty = 2)
        lines(density(resid(lm(jjas_1d ~ gmst, df))))
        overlay_dnorm(resid(lm(jjas_1d ~ gmst, df)), x = seq(-1,1,0.01), lty = 2)
        legend("topright", legend = c("1day", "7day", "Monthly"), lty = 1, col = c("black", "blue2", "red3"))
    
        mtext(mdl, outer = T, font = 2)
    }; dev.off()
}

## Model fitting

### GMST only

In [138]:
nsamp <- 1000
vars <- c("jjas_1d", "jjas_7d", "jjas_m")

for (fnm in list.files("ts", pattern = "sst", full.names = T)) {
    
    mnm <- strsplit(gsub(".csv","",fnm),"_")[[1]][3]

    gmst <- load_ts(list.files("ts", pattern = paste0("SGSAT2023.",mnm), full.names = T), col.names = c("year", "gmst"))
    nino <- read.csv(paste0("ts/cmip6_nino34det_",mnm,"_r1i1p1f1.csv"), col.names = c("year", "nino34"))
    df <- merge(merge(gmst, nino), read.csv(fnm))
    
    cov_2024 <- gmst[gmst$year == 2024,,drop = F]
    cov_cf <- rbind("pi" = cov_2024 - 1.3)
    
    for (varnm in vars) {
        
        mdl <- fit_ns("norm", "shift", df, varnm, "gmst", lower = F, ev_year = 2024)
        
        # get bootstrapped intervals
        res_fnm <- paste0("res/cmip6_sst-",varnm,"_gmst-only_",mnm,".csv")
        
        
        res <- boot_ci(mdl, cov_f = cov_2024, cov_cf = cov_cf, nsamp = nsamp)

        # get correlation between Nino3.4 and SSTs
        set.seed(42)
        nino_corr <- c(cor(df$nino34, df[,varnm]), quantile(sapply(1:nsamp, function(i) cor(df[sample(1:nrow(df), replace = T), c("nino34", varnm)])[1,2]), c(0.025, 0.975)))

        write.csv(rbind(res, nino_corr), res_fnm)
    }
}

[1] "WARNING: Event year not in data, no event value recorded"
[1] "WARNING: Event year not in data, no event value recorded"
[1] "WARNING: Event year not in data, no event value recorded"


#### Compare parameter estimates

In [169]:
# load all bootstrapped results
res_all <- abind::abind(sapply(vars, function(varnm) abind::abind(sapply(list.files("res", pattern = paste0("sst-",varnm,"_gmst-only"), full.names = T), function(fnm) {
    read.csv(fnm, row.names = "X")
#     unlist(lapply(rownames(res), function(cnm) setNames(res[cnm,], paste(gsub("_", "-", cnm), c("est", "lower", "upper"), sep = "_"))))
}, simplify = F), rev.along = 0), simplify = F), rev.along = 0)
dimnames(res_all)[[3]] <- sapply(strsplit(gsub(".csv", "", dimnames(res_all)[[3]]), "_"), "[", 5)
                               
vars <- dimnames(res_all)[[3]]
n_vars <- length(vars)

r <- "sigma0" # alpha_gmst; PR; nino_corr

dpi = 360
png(paste0("fig/cmodels/pars_cmip6_sst_",gsub("_","-",r),".png"), height = dpi, width = dpi /3 * n_vars); {
    prep_window(c(1,n_vars), mar = c(4,3,3,1), oma = c(0,0,2,0))

    for (varnm in vars) {
        res <- t(res_all[r,,varnm,])
        ylim = range(pretty(res_all[r,,,]))
        
        n_obs <- nrow(res)
        logaxs = ""
        h = 0

        if(grepl("PR", r)) {
            h = 1
            logaxs = "y"
            ylim = exp(range(pretty(log(res_all["PR",,,]))))
        }
        
        plot(1, type = "n", ylim = ylim, xlim = c(0,n_obs+1), main = varnm, xaxt = "n", xlab = "", ylab = "", log = logaxs)
        axis(1, at = 1:n_obs, labels = rownames(res), las = 2)
        segments(x0 = 1:n_obs, y0 = res[,"X2.5."], y1 = res[,"X97.5."], lwd = 8, col = adjustcolor(c("blue"), alpha = 0.4))
        points(1:n_obs, res[,"est"], pch = 20, cex = 2)
        abline(h = h, lty = 2)
    }
    mtext(paste0("SST ", r), outer = T, font = 2)
}; dev.off()

---
# Potential intensity

## Choose distribution

In [82]:
nc = 2; dpi = 360
for (fnm in list.files("ts", pattern = "pi", full.names = T)) {
    
    mdl = strsplit(gsub(".csv","",fnm),"_")[[1]][3]
    df = read.csv(fnm)
    
    png(paste0("fig/cmodels/dist_",substr(fnm,4,nchar(fnm)-4),".png"), width = dpi*nc, height = dpi); {
        prep_window(c(1,nc), oma = c(0,0,2,0))

        plot(df$year, df$jjas_1d, type = "l", ylab = "Mean potential intensity", xlab = "", main = "Annual time series")
        lines(df$year, df$jjas_7d, col = "blue2")
        lines(df$year, df$jjas_7d, col = "red3")
        legend("topleft", legend = c("1day", "7day", "Monthly"), lty = 1, col = c("black", "blue2", "red3"))
        
        # overlay with a normal distribution - looks OK in most cases
        plot(density(df$jjas_m), col = "red3", xlab = "Mean potential intensity", main = "Distribution of PI", ylim = c(0,0.15))
        ndist(df$jjas_m, col = "red3", lty = 2)
        lines(density(df$jjas_7d), col = "blue2")
        ndist(df$jjas_7d, col = "blue2", lty = 2)
        lines(density(df$jjas_1d))
        ndist(df$jjas_1d, lty = 2)
        legend("topleft", legend = c("1day", "7day", "Monthly"), lty = 1, col = c("black", "blue2", "red3"))

        mtext(mdl, outer = T, font = 2)
    }; dev.off()
}

## Model fitting

### GMST only

In [171]:
nsamp <- 1000
vars <- c("jjas_1d", "jjas_7d", "jjas_m")

for (fnm in list.files("ts", pattern = "pi", full.names = T)) {
    
    mnm <- strsplit(gsub(".csv","",fnm),"_")[[1]][3]

    gmst <- load_ts(list.files("ts", pattern = paste0("SGSAT2023.",mnm), full.names = T), col.names = c("year", "gmst"))
    nino <- read.csv(paste0("ts/cmip6_nino34det_",mnm,"_r1i1p1f1.csv"), col.names = c("year", "nino34"))
    df <- merge(merge(gmst, nino), read.csv(fnm))
    
    cov_2024 <- gmst[gmst$year == 2024,,drop = F]
    cov_cf <- rbind("pi" = cov_2024 - 1.3)
    
    for (varnm in vars) {
        
        mdl <- fit_ns("norm", "shift", df, varnm, "gmst", lower = F, ev_year = 2024)
        
        # get bootstrapped intervals
        res_fnm <- paste0("res/cmip6_pi-",varnm,"_gmst-only_",mnm,".csv")
        
        
        res <- boot_ci(mdl, cov_f = cov_2024, cov_cf = cov_cf, nsamp = nsamp)

        # get correlation between Nino3.4 and SSTs
        set.seed(42)
        nino_corr <- c(cor(df$nino34, df[,varnm]), quantile(sapply(1:nsamp, function(i) cor(df[sample(1:nrow(df), replace = T), c("nino34", varnm)])[1,2]), c(0.025, 0.975)))

        write.csv(rbind(res, nino_corr), res_fnm)
    }
}

#### Compare parameter estimates

In [187]:
# load all bootstrapped results
res_all <- abind::abind(sapply(vars, function(varnm) abind::abind(sapply(list.files("res", pattern = paste0("pi-",varnm,"_gmst-only"), full.names = T), function(fnm) {
    read.csv(fnm, row.names = "X")
#     unlist(lapply(rownames(res), function(cnm) setNames(res[cnm,], paste(gsub("_", "-", cnm), c("est", "lower", "upper"), sep = "_"))))
}, simplify = F), rev.along = 0), simplify = F), rev.along = 0)
dimnames(res_all)[[3]] <- sapply(strsplit(gsub(".csv", "", dimnames(res_all)[[3]]), "_"), "[", 5)
                               
mnames <- dimnames(res_all)[[3]]
n_vars <- length(mnames)
dpi = 360

In [189]:
for (r in c("alpha_gmst", "sigma0", "PR", "nino_corr")) {
    png(paste0("fig/cmodels/pars_cmip6_pi_",gsub("_","-",r),".png"), height = dpi, width = dpi /3 * n_vars); {
        prep_window(c(1,n_vars), mar = c(4,3,3,1), oma = c(0,0,2,0))

        for (varnm in vars) {
            res <- t(res_all[r,,varnm,])
            ylim = range(pretty(res_all[r,,,]))

            n_obs <- nrow(res)
            logaxs = ""
            h = 0

            if(grepl("PR", r)) {
                h = 1
                logaxs = "y"
                ylim = exp(range(pretty(log(res_all["PR",,,]))))
            }

            plot(1, type = "n", ylim = ylim, xlim = c(0,n_obs+1), main = varnm, xaxt = "n", xlab = "", ylab = "", log = logaxs)
            axis(1, at = 1:n_obs, labels = rownames(res), las = 2)
            segments(x0 = 1:n_obs, y0 = res[,"X2.5."], y1 = res[,"X97.5."], lwd = 8, col = adjustcolor(c("blue"), alpha = 0.4))
            points(1:n_obs, res[,"est"], pch = 20, cex = 2)
            abline(h = h, lty = 2)
        }
        mtext(paste0("PI ", r), outer = T, font = 2)
    }; dev.off()
}

ERROR: Error in res_all[r, , varnm, ]: subscript out of bounds


In [190]:
res

,est,X2.5.,X97.5.
jjas_1d,-0.1339951,-0.2464460,-0.009782186
jjas_7d,-0.1438395,-0.2592055,-0.023516149
jjas_m,-0.1619971,-0.2798170,-0.035050257
